**----LOAD THE PRETRAINED GLOVE MODEL AND BUILD THE DICTONARY FOR WORD INDEX AND VICE-VERSA---**

In [1]:
import numpy as np
#file_path = "F:/Study_and_Others/LCT/Trento/study_material/Computational_Linguistics/project_idea_/pre_trained_model/glove.42B.300d"
embedding_index = {}
i=0
word2idx={}
idx2word={}
#with open('glove.42B.300d.txt', encoding='utf-8') as f:
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:])   
        #embedding_index[word] =np.array(np.float64(coefs))
        embedding_index[word] =coefs
        word2idx[word]=i
        idx2word[i]=word
        i=i+1

# create embedding matrix
# word2idx={}
# idx2word={}
# i=0
#matrix=[]    #----------------------------WE ARE NOT BUILDING THE MATRIX AS IT IS TAKING HUGE TIME------------------
# for word, v in embedding_index.items():
#         word2idx[word]=i
#         idx2word[i]=word
#         i=i+1
        #matrix.append(embedding_index[word].astype(np.float64))

In [2]:
#type(matrix[0][0])

**--------Glove_Vector FUNCTION TAKES A WORD AS INPUT AND RETURNS THE CORRESPONDING WORD VECTOR-------**

In [3]:
def Glove_Vector(word):
    return embedding_index[word]

**------ exists FUNCTION TAKES A WORD AS PARAMETER AND RETURNS IF THE WORD EXIXTS IN THE DICTIONARY OR NOT---**

In [4]:
def exists(word):
    try :
        v=embedding_index[word]
        return 1
    except :
        return 0

In [5]:
import import_ipynb
import pandas as pd
from Data_Loader import MEN_Dataloader, CC_Dataloader, gender_Dataloader
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics

importing Jupyter notebook from Data_Loader.ipynb


**--------SEMANTIC RELATEDNESS------**

In [6]:
wordList_1, wordList2, gold_std = MEN_Dataloader()
cos_sim=[]
for w1, w2 in zip(wordList_1, wordList2):
    v1 = Glove_Vector(w1).reshape(1,-1)
    v2 = Glove_Vector(w2).reshape(1,-1)
    sim = float(cosine_similarity(v1, v2))
    cos_sim.append(sim)

In [7]:
similarity_df=pd.DataFrame({'Gold': gold_std, 'Cos Sim': cos_sim})

print("---- Experiment 2: MEN dataset-------")
print("Pearson Co-relation: \n", similarity_df.corr(method='pearson') )
print("\nSpearman Co-relation: \n", similarity_df.corr(method='spearman'))

---- Experiment 2: MEN dataset-------
Pearson Co-relation: 
              Gold   Cos Sim
Gold     1.000000  0.696712
Cos Sim  0.696712  1.000000

Spearman Co-relation: 
              Gold   Cos Sim
Gold     1.000000  0.693187
Cos Sim  0.693187  1.000000


**------ N_Nearest_Neighbour TAKES A WORD-VECTOR AS INPUT AND RETURNS N- NEAREST NEIGHBOURS -----**

In [8]:
def N_Nearest_Neighbhour(v1, n):
    nearest_neighbours={}
    v1=v1.reshape(1,-1)
    for word, v in embedding_index.items():
        v2= v.reshape(1,-1)
        dis = 1 - cosine_similarity(v1, v2)
        if len(nearest_neighbours)<n:
            nearest_neighbours[word]= dis
        else:
            max_dis=0
            farthest=""
            for neigh_x, dis_x in nearest_neighbours.items():
                if dis_x> max_dis: 
                    farthest=neigh_x
                    max_dis=dis_x
            if dis<max_dis:
                del nearest_neighbours[farthest]
                nearest_neighbours[word]=dis
    neighbours=[]
    for neigh_x, dis_x in nearest_neighbours.items():
        neighbours.append(neigh_x)
    return neighbours

In [9]:
def AN_Dataloader():
    dataset= open('./dataset/analogy_test_reduced_2.txt')
    word_list_1=[]
    word_list_2=[]
    word_list_3=[]
    word_list_4=[]
    for line in dataset:
        if line.split(' ')[0]==':':
            continue
        w1 = line.split(' ')[0]  
        w2 = line.split(' ')[1] 
        w3 = line.split(' ')[2]  
        w4 = line.split(' ')[3]
        word_list_1.append(w1)
        word_list_2.append(w2)
        word_list_3.append(w3)
        word_list_4.append(w4)
    return word_list_1, word_list_2, word_list_3, word_list_4

**--------------ANALOGY TESTING---------**

In [10]:
def analogy_test():
    word_list_1, word_list_2, word_list_3, word_list_4 = AN_Dataloader()
    total_test = len(word_list_1)
    missed = successful = unsuccessful = 0
    for w1, w2, w3, w4 in zip(word_list_1, word_list_2, word_list_3, word_list_4):
        w1=w1.lower()
        w2=w2.lower()
        w3=w3.lower()
        w4=w4.lower().strip()
        #print(w1, w2, w3, w4, len(w4)) #------REMOVE THIS------
        #print(exists(w1), exists(w2), exists(w3), exists(w4))  #------REMOVE THIS------
        if exists(w1)==1 and exists(w2)==1 and exists(w3)==1 and exists(w4)==1 :
            #v1=Glove_Vector(w1)
            #v2=Glove_Vector(w2)
            #v3=Glove_Vector(w3)
            v1= np.array([np.float64(x) for x in Glove_Vector(w1)])
            v2= np.array([np.float64(x) for x in Glove_Vector(w2)])
            v3= np.array([np.float64(x) for x in Glove_Vector(w3)])
            v= v2-v1+v3
            nearest_neighbours = N_Nearest_Neighbhour(v, 5)
            #print(nearest_neighbours) #------REMOVE THIS------
            if w4 in nearest_neighbours: 
                successful += 1
            else:
                unsuccessful +=1
        else:
            missed += 1
    return total_test, missed, successful, unsuccessful

**----------CONCEPT CATEGORIZATION --------------**

In [11]:
from sklearn.cluster import KMeans
from sklearn import metrics
test_word_embeddings = np.zeros((44, 100)) #---------- change the dimension (100 or 300) based on Golve----
word_list, gold_standard_labels = CC_Dataloader()
print(word_list)
index=0
for w in word_list:
    w=w.lower()
    missed=0
    if exists(w):
        v= np.array([np.float64(x) for x in Glove_Vector(w)])
        test_word_embeddings[index] = v
        index+=1
    else:
        missed+=1

print(len(test_word_embeddings))   
kmeans = KMeans(init="random", n_clusters=6, n_init=10, max_iter=50)
result = kmeans.fit(test_word_embeddings) 
clustering_labels = result.labels_  # get the cluster ID assigned to each word embedding
print(gold_standard_labels)
print(clustering_labels)
contingency_matrix = metrics.cluster.contingency_matrix(gold_standard_labels, clustering_labels) 


max_each_cluster = np.amax(contingency_matrix, axis=0)  
total_number_datapoints = np.sum(contingency_matrix)  

purity = np.sum(max_each_cluster) / total_number_datapoints  
print("Contingency Matrix:\n", contingency_matrix)  
print("Purity:", purity)

['chicken', 'eagle', 'duck', 'swan', 'owl', 'penguin', 'peacock', 'dog', 'elephant', 'cow', 'cat', 'lion', 'pig', 'snail', 'turtle', 'cherry', 'banana', 'pear', 'pineapple', 'mushroom', 'corn', 'lettuce', 'potato', 'onion', 'bottle', 'pencil', 'pen', 'cup', 'bowl', 'scissors', 'kettle', 'knife', 'screwdriver', 'hammer', 'spoon', 'chisel', 'telephone', 'boat', 'car', 'ship', 'truck', 'rocket', 'motorcycle', 'helicopter']
44
['bird', 'bird', 'bird', 'bird', 'bird', 'bird', 'bird', 'groundAnimal', 'groundAnimal', 'groundAnimal', 'groundAnimal', 'groundAnimal', 'groundAnimal', 'groundAnimal', 'groundAnimal', 'fruitTree', 'fruitTree', 'fruitTree', 'fruitTree', 'vegetable', 'vegetable', 'vegetable', 'vegetable', 'vegetable', 'tool', 'tool', 'tool', 'tool', 'tool', 'tool', 'tool', 'tool', 'tool', 'tool', 'tool', 'tool', 'tool', 'vehicle', 'vehicle', 'vehicle', 'vehicle', 'vehicle', 'vehicle', 'vehicle']
[5 0 0 0 0 0 0 0 0 0 0 0 0 5 5 5 5 5 5 5 5 1 5 1 1 2 2 1 1 2 5 2 2 2 1 2 4
 3 3 3 3 3 3 3]

**---- GENDER BIASES WITH T-TEST (P-VALUES)-----

In [12]:
from scipy.stats import ttest_ind
word_list= gender_Dataloader()
male_list=["he", 'him', 'himself', 'male', 'boy', 'man', 'masculine']
female_list=["she", 'her', 'herself', 'female', 'girl', 'woman', 'feminine']
biased=0
non_biased=0
for word in word_list:
    list1=[]
    list2=[]
    if exists(word):
        v=np.array([np.float64(x) for x in Glove_Vector(word)]).reshape(1,-1)
        m=0
        f=0
        for him, her in zip(male_list, female_list):
            if exists(him) and exists(her):
                v1= np.array([np.float64(x) for x in Glove_Vector(him)]).reshape(1,-1)
                v2= np.array([np.float64(x) for x in Glove_Vector(her)]).reshape(1,-1)
                sim1=(cosine_similarity(v, v1)*100)[0][0]
                sim2=(cosine_similarity(v, v2)*100)[0][0]
                if(sim1<0):
                    sim1*=(-1)
                if(sim2<0):
                    sim2*=(-1)
                dif=sim1-sim2
                abs_dif=dif
                if(dif<0):
                    abs_dif=dif*(-1)
                if(abs_dif>0.5 and dif>0):
                    m+=1
                if(abs_dif>0.5 and dif<0):
                    f+=1
                list1.append(sim1)
                list2.append(sim2)
        if(abs(m-f)>1):
            biased+=1
        else:
            non_biased+=1
        print(m, f, ttest_ind(list1, list2).pvalue)
print(biased, non_biased)

1 5 0.7243122183691963
5 2 0.548678320731334
2 5 0.9602976273289018
5 2 0.8540946924018984
3 4 0.9085079600345947
6 1 0.18545361096633628
3 4 0.89304993551109
5 1 0.5069952435814811
0 7 0.20907939836780545
1 4 0.6478614019450926
5 2 0.8334469092111655
2 4 0.6895767092787213
0 7 0.016841947602898463
5 2 0.753006966861041
3 4 0.9837106148747252
0 7 0.1213822684394614
4 3 0.9792941104821212
1 6 0.06259014027963461
1 4 0.5100226138345756
3 2 0.9274883972980265
2 3 0.8196467430966936
1 6 0.3551981265956724
4 1 0.449971114410177
4 3 0.7080879359499401
2 5 0.5711518486978115
2 5 0.11793318866057048
3 3 0.974139894788099
2 5 0.3436642442696589
4 3 0.6643813392722151
3 2 0.9298261518128617
3 4 0.9688078183078943
0 7 0.02078667239753196
0 7 0.11879089751480836
3 3 0.8317454855163424
1 6 0.08843082650240279
2 5 0.7290222222608551
5 1 0.21068827728674538
1 5 0.6941529973530988
4 3 0.8051938342009759
3 4 0.9407623179892812
2 3 0.99898316760423
2 5 0.5474780290388841
5 2 0.45862308560115317
6 1 0.25

In [13]:
total_test, missed, successful, unsuccessful = analogy_test()


Accuracy1:  0.68


NameError: name 'accuracy2' is not defined

In [14]:
accuracy1= successful/(successful+unsuccessful)
accuracy2= successful/(successful+unsuccessful+missed)
print("Accuracy1: ", accuracy1)
print("Accuracy2: ", accuracy2)
print(successful, unsuccessful, missed)

Accuracy1:  0.68
Accuracy2:  0.68
34 16 0


**----- This part is just rough code-----